In [7]:
import numpy as np
import time
from scipy.interpolate import interp1d
from numba import njit

In [8]:
@njit
def interp1d_numba(x, y, x_new):
    """
    Linear interpolation function using NumPy and Numba.
    
    Parameters:
        x (array-like): 1-D array of x-coordinates of data points.
        y (array-like): 1-D array of y-coordinates of data points.
        x_new (array-like): 1-D array of x-coordinates for which to interpolate.
        
    Returns:
        array-like: 1-D array of interpolated values corresponding to x_new.
    """
    x = np.asarray(x)
    y = np.asarray(y)
    x_new = np.asarray(x_new)
    
    # Sorting based on x
    sorted_indices = np.argsort(x)
    x_sorted = x[sorted_indices]
    y_sorted = y[sorted_indices]
    
    # Find indices of the nearest points for interpolation
    idx = np.searchsorted(x_sorted, x_new, side='right') - 1
    idx = np.clip(idx, 0, len(x_sorted) - 2)
    
    # Compute weights for interpolation
    x0 = x_sorted[idx]
    x1 = x_sorted[idx + 1]
    y0 = y_sorted[idx]
    y1 = y_sorted[idx + 1]
    weights = (x_new - x0) / (x1 - x0)
    
    # Perform linear interpolation
    interpolated_values = y0 + (y1 - y0) * weights
    return interpolated_values


In [9]:
# Sample data for interpolation
x = np.array([0, 1, 2, 3, 4])
y = np.array([0, 2, 4, 6, 8])

# New x-coordinates for interpolation
x_new = np.array([0.5, 1.5, 2.5, 3.5])

# Interpolate using the custom function
result = interp1d_numba(x, y, x_new)
print(result)  # Output: [1. 3. 5. 7.]

[1. 3. 5. 7.]


In [17]:
# Sample data for interpolation
x = np.linspace(0, 100, 1000)
y = np.sin(x)

# New x-coordinates for interpolation
x_new = np.linspace(0, 100, 100000)

# Using scipy.interpolate.interp1d()
start_time = time.time()
f_scipy = interp1d(x, y, kind='linear')
result_scipy = f_scipy(x_new)
scipy_time = time.time() - start_time

# Using custom interp1d_numba function
start_time = time.time()
result_custom = interp1d_numba(x, y, x_new)
custom_time = time.time() - start_time

# Compare the results to make sure they are similar
print(np.allclose(result_scipy, result_custom))

# Print the execution times
print("Scipy interpolation time:", scipy_time)
print("Custom Numba interpolation time:", custom_time)


True
Scipy interpolation time: 0.0011532306671142578
Custom Numba interpolation time: 0.002240896224975586
